In [1]:
import pandas as pd

from notebooks.ford.decode import search, print_breakdown


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

df_nhtsa = search(
  searches=['Escape', 'Bronco Sport'],
  # searches=['Explorer'],
  # searches=['F-150'],
  include_openpilot=True,
  include_police=False,
)

print()
print_breakdown(df_nhtsa)

Loaded 334 VINs (filter_comment='Escape', include_openpilot=True, skipped=0)


Loaded AsBuilt data for 334 VINs
Loaded 131 VINs (filter_comment='Bronco Sport', include_openpilot=True, skipped=1)


Loaded AsBuilt data for 131 VINs


Decoding VINs: 100%|██████████| 465/465 [00:00<00:00, 17335.99it/s]

Decoded 465 VINs

Model         ModelYear  Series         
Bronco Sport  2021       BADLANDS            4
                         BASE                6
                         BIG BEND           10
                         FIRST EDITION       3
                         OUTER BANKS         3
              2022       BADLANDS            4
                         BIG BEND            7
                         OUTER BANKS         4
              2023       BADLANDS           16
                         BIG BEND           20
                         HERITAGE            6
                         OUTER BANKS        16
              2024       BIG BEND           19
                         OUTER BANKS        13
Escape        2015       SE                  1
              2018       S                   2
                         Titanium            1
              2019       S                  10
                         SE                 10
                         SEL                10
 

In [2]:
SKIP = [
  'AirBagLocKnee',
  'AirBagLocSide',
  'BodyClass',
  'DisplacementCC',
  'DisplacementCI',
  'DisplacementL',
  'EngineCylinders',
  'GVWR',
  'LowerBeamHeadlampLightSource',
  'Make',
  'MakeID',
  'Manufacturer',
  'ManufacturerId',
  'ModelID',
  'NCSABodyType',
  'NCSAMake',
  'NCSAModel',
  'PlantCity',
  'PlantCompanyName',
  'PlantCountry',
  'PlantState',
  'Seats',
  'WheelSizeFront',
  'WheelSizeRear',
  'VehicleType',
  'VIN',
  'VehicleDescriptor',
  'WheelSizeFront',
  'WheelSizeRear',
]

KEEP = [
  # 'DisplacementL',
  'DriveType',
  # 'ElectrificationLevel',
  # 'FuelTypePrimary',
  # 'FuelTypeSecondary',
  'Model',
  'Series',
  'Trim',
]

properties = {}

for col in KEEP:
  if col not in df_nhtsa.columns:
    print(f'WARNING: {col} not in df_nhtsa.columns')

for col in df_nhtsa.columns:
  if col in SKIP:
    continue

  property_values = set(df_nhtsa[col].unique())
  if col not in KEEP:
    if '' in property_values:
      continue
    if len(property_values) == 1:
      continue

  properties[col] = property_values

df_nhtsa.drop(columns=['FuelTypePrimary', 'FuelTypeSecondary'], inplace=True, errors='ignore')
properties.pop('FuelTypePrimary', None)
properties.pop('FuelTypeSecondary', None)

properties

{'DriveType': {'', '2WD', '4WD'},
 'ElectrificationLevel': {'FHEV', 'HEV', 'ICE', 'PHEV'},
 'Model': {'Bronco Sport', 'Escape'},
 'ModelYear': {2015, 2018, 2019, 2020, 2021, 2022, 2023, 2024},
 'Series': {'Active',
  'BADLANDS',
  'BASE',
  'BIG BEND',
  'Base',
  'FIRST EDITION',
  'HERITAGE',
  'OUTER BANKS',
  'PHEV',
  'Platinum',
  'S',
  'SE',
  'SEL',
  'ST Line',
  'ST Line Elite',
  'ST Line Premium',
  'ST Line Select',
  'Titanium'}}

### Combine NHTSA and Ford AsBuilt Data

We fetch the factory part numbers (software and hardware) from the Ford AsBuilt data and combine it with the NHTSA data.

In [3]:
from panda.python.uds import DATA_IDENTIFIER_TYPE

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.ecu import FordEcu
from notebooks.ford.settings import VehicleSetting, VehicleSettings


df_fw = df_nhtsa.copy()

# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
)


ecus = {
  'abs': FordEcu.AntiLockBrakeSystem,
  # 'engine': FordEcu.PowertrainControlModule,
  'eps': FordEcu.PowerSteeringControlModule,
  'fwdCamera': FordEcu.ImageProcessingModuleA,
  'fwdRadar': FordEcu.CruiseControlModule,
}


def get_ecu_identifier(ecu: FordEcu, identifier: int):
  def apply(row):
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return ''
    return data.get_identifier(ecu, identifier)
  return apply


def get_setting(setting: VehicleSetting):
  def apply(row):
    ecu = setting.ecu
    ecu_name = next(filter(lambda name: ecus[name] == ecu, ecus.keys()))
    ecu_part = row[f'{ecu_name}_part']
    if not ecu_part:
      return 'Missing ECU'
    # TODO: detect ECUs which we can't read
    # if ecu_part[0] in ('M', 'N', 'O', 'P'):
    #   return 'CAN FD'
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return 'Missing ECU'
    return data.get_setting_value(setting)
  return apply


# Add the ECU identifiers
for name, ecu in ecus.items():
  df_fw[f'{name}_fw'] = df_fw.apply(
    get_ecu_identifier(
      ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER
    ),
    axis=1,
  )
  df_fw[f'{name}_part'] = df_fw.apply(get_ecu_identifier(ecu, 0xF111), axis=1)


# Determine platform (pre-Q3, Q3 or Q4)
def get_platform(row):
  camera_pn = row['fwdCamera_part']
  core = camera_pn.split('-')[1] if camera_pn else None
  if not core:
    return 'Missing ECU'

  platform = {
    '14F403': 'Q3',
    '14G025': 'pre-Q3',  # guess, seen on 2020 Fusion/Mondeo
    '14H107': 'Q4',
  }.get(core)

  if platform:
    return platform
  assert False, f'Unhandled platform for {row["ModelYear"]} {row["Model"]} {camera_pn=}'


df_fw['Platform'] = df_fw.apply(get_platform, axis=1)


# Add settings
settings = {
  # TODO: read this from multiple modules to check that it's consistent
  'acc': VehicleSettings.ipma_enable_adaptive_cruise,
  'lca': VehicleSettings.ipma_enable_traffic_jam_assist,
}
for name, setting in settings.items():
  df_fw[f'code_{name}'] = df_fw.apply(get_setting(setting), axis=1)


# Clear settings if platform is Q4
# TODO: simplify
for setting in settings.keys():
  df_fw[f'code_{setting}'] = df_fw.apply(
    lambda row: '' if row['Platform'] == 'Q4' else row[f'code_{setting}'],
    axis=1,
  )


# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
  errors='ignore',
)


# Merge 'Series' and 'Trim' properties (they should be mutually exclusive)
if 'Trim' in properties and 'Series' in properties:
  def get_series_or_trim(row):
    series, trim = row['Series'], row['Trim']
    if series == 'F-Series':
      return trim
    elif series and trim:
      return f'{series} ({trim})'
      # raise ValueError(f'{row["VIN"]} Both Series and Trim are set: {series} and {trim}')
    return series or trim

  df_fw['Trim'] = df_fw.apply(get_series_or_trim, axis=1)
  df_fw.drop(columns=['Series'], inplace=True)
  properties.pop('Series', None)
elif 'Series' in properties:
  df_fw.rename(columns={'Series': 'Trim'}, inplace=True)
  properties['Trim'] = properties.pop('Series')


# Apply filters
constants = {
  # 'ModelYear': '2020',
  # 'DisplacementL': '1.5',
  # 'DriveType': '4x2',
  # 'EngineCylinders': '3',
  # 'Series': 'Titanium',
}
for col, value in constants.items():
  df_fw = df_fw[df_fw[col] == value]
  df_fw.drop(columns=[col], inplace=True)


# Drop columns that are all empty
df_fw = df_fw.loc[:, (df_fw != '').any(axis=0)]


# Drop the VIN
df_fw.drop(columns=['VIN'], inplace=True, errors='ignore')


# Sort by columns
main_columns = []
if 'ModelYear' in df_fw.columns:
  main_columns.append('ModelYear')
if 'Series' in df_fw.columns:
  main_columns.append('Series')

extra_columns = list(set(properties.keys()) - set(main_columns))

# + [f'code_{name}' for name in settings.keys()] 
df_fw.sort_values(
  by=main_columns + extra_columns + [f'{name}_fw' for name in ecus.keys()],
  ascending=False,
  inplace=True,
  ignore_index=True,
)

# Delete 'Series' column
df_fw.drop(columns=['Series'], inplace=True, errors='ignore')


# Drop columns that are all the same
# df_fw = df_fw.loc[:, df_fw.apply(pd.Series.nunique) != 1]


# Add asterisks to column name where the value is the same for all rows
# df_fw.rename(
#   columns={
#     col: f'*{col}' if len(set(df_fw[col].unique())) == 1 else col
#     for col in df_fw.columns
#   },
#   inplace=True,
# )

# Remove duplicate rows

count = len(df_fw)
df_fw.drop_duplicates(inplace=True)
print(f'Removed {count - len(df_fw)} duplicate rows')


df_fw

Removed 256 duplicate rows


,DriveType,ElectrificationLevel,Model,ModelYear,Trim,abs_fw,abs_part,eps_fw,eps_part,fwdCamera_fw,fwdCamera_part,fwdRadar_fw,fwdRadar_part,Platform,code_acc,code_lca
0,4WD,ICE,Escape,2024,ST Line Premium,PZ1C-2D053-EL,PZ1C-14F065-EA,PZ11-14D003-EA,PZ11-14F079-DA,PJ6T-14H102-ABL,PJ6T-14H107-ECB,ML3T-14D049-AL,ML3T-14F089-AH,Q4,,
1,4WD,ICE,Escape,2024,ST Line,PZ1C-2D053-EL,PZ1C-14F065-EA,PZ11-14D003-EA,PZ11-14F079-DA,PJ6T-14H102-ABL,PJ6T-14H107-BBB,ML3T-14D049-AL,ML3T-14F089-AH,Q4,,
2,4WD,ICE,Escape,2024,Active,PZ1C-2D053-EL,PZ1C-14F065-EA,PZ11-14D003-EA,PZ11-14F079-DA,PJ6T-14H102-ABL,PJ6T-14H107-BAB,,,Q4,,
5,2WD,ICE,Escape,2024,Active,PZ1C-2D053-EL,PZ1C-14F065-EA,PZ11-14D003-EA,PZ11-14F079-DA,PJ6T-14H102-ABL,PJ6T-14H107-EBB,ML3T-14D049-AL,ML3T-14F089-AH,Q4,,
6,2WD,ICE,Escape,2024,Active,PZ1C-2D053-EL,PZ1C-14F065-EA,PZ11-14D003-EA,PZ11-14F079-DA,PJ6T-14H102-ABL,PJ6T-14H107-BAB,,,Q4,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,2WD,ICE,Escape,2019,SE,GV61-14C036-CL,GV61-14F067-CD,HV6T-14C217-AC,JV6T-14C262-AB,,,,,Missing ECU,Missing ECU,Missing ECU
451,2WD,ICE,Escape,2019,S,GV61-14C036-CL,GV61-14F067-CD,HV6T-14C217-AC,JV6T-14C262-AB,,,,,Missing ECU,Missing ECU,Missing ECU
461,2WD,ICE,Escape,2018,Titanium,GV61-14C036-CJ,GV61-14F067-CC,HV6T-14C217-AA,JV6T-14C262-AB,,,,,Missing ECU,Missing ECU,Missing ECU
462,2WD,ICE,Escape,2018,S,GV61-14C036-CJ,GV61-14F067-CC,HV6T-14C217-AA,JV6T-14C262-AB,,,,,Missing ECU,Missing ECU,Missing ECU


In [4]:
df_fw_analysis = df_fw.copy()

SORTED_TRIM_LEVELS = [
  'Base',
  # Escape, Focus, Fusion
  'S',
  'SE',
  'SEL',
  'Titanium',
  'Active',
  'ST Line',
  'ST Line Select',
  'ST Line Elite',
  'ST Line Premium',
  'PHEV',
  # Bronco Sport
  'BIG BEND',
  'OUTER BANKS',
  'BADLANDS',
  'HERITAGE',
  # Explorer
  'XL',
  'XLT',
  'Limited',
  'ST',
  'Platinum',
  'Timberline',
  'King Ranch',
  # Aviator
  'Standard',
  'Reserve',
  'Livery',
  'Grand Touring',
]

def sort_trim_level(trim):
  if trim in SORTED_TRIM_LEVELS:
    return SORTED_TRIM_LEVELS.index(trim)
  return 1000

for ecu in reversed(ecus.keys()):
  print()
  print(f'# Ecu.{ecu}')

  fw_groups = df_fw_analysis \
    .drop(
      columns=[f'{name}_fw' for name in ecus.keys() if name != ecu] + [f'{name}_part' for name in ecus.keys() if name != ecu],
    ) \
    .rename(
      columns={
        f'{ecu}_fw': 'fw',
        f'{ecu}_part': 'part',
      },
    ) \
    .groupby(
      # by=['part'],
      # by=['fw'],
      by=['part', 'fw'],
      dropna=False,
    )
  # print(fw_groups.nunique().to_string())

  print(fw_groups.agg(lambda x: set(x)).to_string())

  # if len(fw_groups) > 1:
  #   for name, group in fw_groups:
  #     print(name)
  #     # print(group.groupby(
  #     #   by=['ModelYear', 'Model', 'Series'] + list(set(group.columns) - {'ModelYear', 'Model', 'Series', 'code_acc', 'code_lca', 'part', 'fw'}) + ['code_acc', 'code_lca'],
  #     #   dropna=False,
  #     # ).count().to_string())

  #     # drive_type = group["DriveType"].unique()
  #     # drive_type = "Both" if "4WD" in drive_type and "RWD" in drive_type else "Yes" if "4WD" in drive_type else "No"

  #     print(f'  ACC?          : {", ".join(group["code_acc"].unique())}')
  #     print(f'  LCA?          : {", ".join(group["code_lca"].unique())}')
  #     print(f'  Model         : {", ".join(group["Model"].unique())}')
  #     print(f'  ModelYear     : {", ".join(sorted(group["ModelYear"].astype(str).unique()))}')
  #     print(f'  Trim          : {", ".join(sorted(group["Trim"].unique(), key=sort_trim_level))}')
  #     # print(f'  DisplacementL : {", ".join(map(str, sorted(group["DisplacementL"].unique())))}')
  #     print(f'  DriveType     : {", ".join(group["DriveType"].unique())}')
  #     print(f'  Hybrid?       : {", ".join(group["ElectrificationLevel"].unique())}')
  #     print(f'  Platform      : {", ".join(group["Platform"].unique())}')

  #     print()

  print()



# Ecu.fwdRadar
                                  DriveType    ElectrificationLevel                   Model                                         ModelYear                                                                                                                           Trim               Platform              code_acc                   code_lca
part           fw                                                                                                                                                                                                                                                                                                                                   
                               {4WD, , 2WD}  {ICE, HEV, FHEV, PHEV}  {Escape, Bronco Sport}  {2018, 2019, 2020, 2021, 2022, 2023, 2024, 2015}  {Base, Active, BASE, FIRST EDITION, SE, OUTER BANKS, ST Line, SEL, Titanium, BADLANDS, ST Line Select, S, BIG BEND, HERITAGE}  {Q4, Missing ECU, Q3}  {, Missin